# Hébergement local de modèles génératifs


## 1) Installation & Import

On installe/importe ce qui est nécessaire : 
- `requests` pour les appels HTTP bruts,
- `openai` version 1.0.0+,
- `semantic-kernel` si on veut tester SK,
- d’autres libs selon besoin (json, time, etc.).



In [ ]:
%pip install --upgrade requests aiohttp openai semantic-kernel python-dotenv anyio httpx httpcore


import os
import requests
import time
import json
import getpass
import openai


print("Importations OK.")

^C
Note: you may need to restart the kernel to use updated packages.
Importations OK.


## Journalisation colorée

Nous allons utiliser un logger (via le module `logging`) configuré avec un
**ColorFormatter** pour afficher les messages en couleur dans la console ou la
sortie de Jupyter :

- Les **informations** et étapes réussies apparaîtront en vert (niveau `INFO`).
- Les **erreurs** seront en rouge (niveau `ERROR`).
- Les avertissements (`WARNING`) ou messages de debug (`DEBUG`) auront également leurs couleurs.


In [ ]:
import logging
class ColorFormatter(logging.Formatter):
    """
    Un formatter coloré pour rendre les logs plus lisibles.
    """
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record: logging.LogRecord) -> str:
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Local Llama")
logger.setLevel(logging.DEBUG)  # Peut être paramétré via .env ou variable

if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        fmt="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Configuration initiale terminée.")

22:44:09 [INFO] Local Llama - Configuration initiale terminée.


## Configuration et définition dynamique des endpoints

Pour simplifier la configuration de nos endpoints (URL d’API, clés d’API, modèles, etc.), nous allons externaliser ces informations dans un fichier `.env` placé à la racine de notre projet ou dans un dossier sécurisé. Copiez le fichier .env.example et renommez le fichier résultant en .env avant de le personnaliser.

### 1) Déclaration dans `.env`

On déclare, par exemple, un premier endpoint dans des variables d’environnement :

```
OPENAI_ENDPOINT_NAME=OpenAI
OPENAI_BASE_URL=https://api.openai.com/v1
OPENAI_API_KEY=sk-abcd1234ABCD1423
OPENAI_CHAT_MODEL_ID=gpt-4o-mini
```

Et si l’on souhaite tester plusieurs endpoints (ex. mini, medium, large), on ajoute un suffixe `_2`, `_3`... :

```
OPENAI_ENDPOINT_NAME_2=local-mini
OPENAI_BASE_URL_2=https://api.mini.yourdomain.com/v1
OPENAI_API_KEY_2=sk-MINI-SECRET-KEY
OPENAI_CHAT_MODEL_ID_2=unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit

OPENAI_ENDPOINT_NAME_3=medium
OPENAI_BASE_URL_3=https://api.medium.text-generation-webui.myia.io/v1
OPENAI_API_KEY_3=sk-MEDIUM-SECRET-KEY
OPENAI_CHAT_MODEL_ID_3=unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit
```

Ainsi, chacun de ces blocs définit un **endpoint** : un service local ou distant OpenAI-compatible (ex. Oobabooga ou vLLM).  

### 2) Lecture et création automatique dans le notebook

Dans le notebook, nous allons **lire** ces variables pour construire la liste `endpoints`. Chacun contient :

- `name` : un label descriptif (ex. `micro`, `mini`, etc.),
- `api_base` : l’URL de base de l’API (ex. `https://api.micro.text-generation-webui.myia.io/v1`),
- `api_key` : la clé API (fournie par votre conteneur ou config),
- `model` (optionnel) : si le modèle n’est pas fourni, nous pourrons interroger `/models` pour récupérer le nom du (ou des) modèle(s) disponibles.

Grâce à cette configuration dynamique, on peut aisément **alterner** entre différents backends (p. ex. Oobabooga ou vLLM) ou **interroger plusieurs endpoints** pour **comparer leurs performances**.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Charge automatiquement toutes les variables du fichier .env


import os

def get_optional_env(var_name, default_value=None):
    """
    Récupère la valeur de la variable d'env `var_name`, 
    ou la valeur par défaut `default_value` si non définie.
    """
    val = os.getenv(var_name)
    if val is None or val.strip() == "":
        return default_value
    return val.strip()

def load_endpoint(index=1):
    """
    Lit un ensemble de variables d'environnement.
    - index=1 => variables : OPENAI_API_KEY, OPENAI_BASE_URL, etc.
    - index>1 => on suffixe : OPENAI_API_KEY_{index}, etc.
    Retourne un dict {name, api_base, api_key, model} ou None si 'api_key' manquant.
    """
    suffix = "" if index == 1 else f"_{index}"

    # Lecture des variables
    api_key = os.getenv(f"OPENAI_API_KEY{suffix}")
    if not api_key:
        return None  # pas de clé => on arrête

    name = get_optional_env(f"OPENAI_ENDPOINT_NAME{suffix}", default_value=f"openai{suffix}")
    base_url = get_optional_env(f"OPENAI_BASE_URL{suffix}", default_value="https://api.openai.com/v1")
    model_id = get_optional_env(f"OPENAI_CHAT_MODEL_ID{suffix}", default_value=None)

    return {
        "name": name,
        "api_base": base_url,
        "api_key": api_key,
        "model": model_id  # On pourra le compléter si None
    }


endpoints = []

# On tente successivement index=1,2,3... jusqu'à ce qu'on ne trouve plus OPENAI_API_KEY_{i}
for i in range(1, 10):  # max 9 endpoints, ajustez si besoin
    ep = load_endpoint(i)
    if ep is None:
        break
    endpoints.append(ep)

# Vérification (simple)
logger.info("=== Endpoints chargés ===")
for e in endpoints:
    logger.info(f"- name={e['name']}, base={e['api_base']}, key=(len={len(e['api_key'])}), model={e['model']}")



22:44:09 [INFO] Local Llama - === Endpoints chargés ===
22:44:09 [INFO] Local Llama - - name=OpenAI, base=https://api.openai.com/v1, key=(len=164), model=gpt-4o-mini
22:44:09 [INFO] Local Llama - - name=Local Model - Micro, base=https://api.micro.text-generation-webui.myia.io/v1, key=(len=32), model=None
22:44:09 [INFO] Local Llama - - name=Local Model - Mini, base=https://api.mini.text-generation-webui.myia.io/v1, key=(len=32), model=None
22:44:09 [INFO] Local Llama - - name=Local Model - Medium, base=https://api.medium.text-generation-webui.myia.io/v1, key=(len=32), model=None
22:44:09 [INFO] Local Llama - - name=Local Model - Large, base=https://api.large.text-generation-webui.myia.io/v1, key=(len=32), model=None


## 3) Inspection des modèles disponibles

Nous allons appeler l'endpoint `/models` de chaque service 
pour récupérer la liste des modèles chargés côté serveur.

Si vous avez mis `model=None` dans la config, vous pourrez automatiquement 
récupérer le `model` à partir des données renvoyées. 


In [8]:

def list_models(api_base, api_key):
    url = f"{api_base}/models"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    try:
        resp = requests.get(url, headers=headers, timeout=20)
        if resp.status_code == 200:
            return resp.json()  # dict
        else:
            return {"error": f"status={resp.status_code}", "text": resp.text}
    except Exception as e:
        return {"error": str(e)}

def update_endpoints_with_model():
    for ep in endpoints:
        logger.info(f"=== {ep['name']} : /models ===")
        start_time = time.time()

        info = list_models(ep["api_base"], ep["api_key"])
        elapsed_time = time.time() - start_time

        # On journalise le JSON complet en DEBUG, pour le diagnostic
        logger.debug(f"Réponse brute: {info}")

        if "error" in info:
            # En cas d'erreur, on logue au niveau ERROR
            logger.error(
                f"Échec de récupération /models "
                f"(endpoint={ep['name']}): {info['error']}, "
                f"texte={info.get('text', '')}"
            )
        else:
            # Succès : on peut afficher le nombre de modèles
            data_models = info.get("data", [])
            logger.info(
                f"Réussite: {len(data_models)} modèle(s) listé(s) "
                f"(endpoint={ep['name']})"
            )

        logger.info(f"  -> Temps de réponse: {elapsed_time:.2f} secondes")

        # On met à jour le modèle si:
        # 1) il n'y a pas d'erreur, et
        # 2) ep["model"] n'est pas déjà défini.
        if "error" not in info and ("model" not in ep or not ep["model"]):
            data_list = info.get("data", [])
            if data_list:
                first_model_id = data_list[0].get("id")
                ep["model"] = first_model_id
                logger.info(f"  -> ep['model'] défini à: {first_model_id}")
            else:
                logger.warning(
                    "  -> Aucune entrée 'data' dans la réponse pour définir ep['model']"
                )

# Appel de la fonction pour tester
update_endpoints_with_model()


09:40:17 [INFO] Local Llama - === OpenAI : /models ===
09:40:19 [DEBUG] Local Llama - Réponse brute: {'object': 'list', 'data': [{'id': 'gpt-4o-mini-audio-preview-2024-12-17', 'object': 'model', 'created': 1734115920, 'owned_by': 'system'}, {'id': 'gpt-4-turbo-2024-04-09', 'object': 'model', 'created': 1712601677, 'owned_by': 'system'}, {'id': 'dall-e-3', 'object': 'model', 'created': 1698785189, 'owned_by': 'system'}, {'id': 'dall-e-2', 'object': 'model', 'created': 1698798177, 'owned_by': 'system'}, {'id': 'o1-2024-12-17', 'object': 'model', 'created': 1734326976, 'owned_by': 'system'}, {'id': 'gpt-4o-audio-preview-2024-10-01', 'object': 'model', 'created': 1727389042, 'owned_by': 'system'}, {'id': 'gpt-4o-audio-preview', 'object': 'model', 'created': 1727460443, 'owned_by': 'system'}, {'id': 'o1-mini-2024-09-12', 'object': 'model', 'created': 1725648979, 'owned_by': 'system'}, {'id': 'gpt-4o-mini-realtime-preview-2024-12-17', 'object': 'model', 'created': 1734112601, 'owned_by': 'sy

## 4) Test brut via `requests.post`

Ce test vérifie le bon fonctionnement de l'endpoint OpenAI-compatible 
sans passer par la librairie `openai`. 

On envoie une requête minimaliste en JSON, 
puis on affiche la réponse brute.


In [9]:
def test_brut_endpoints():
    """Test brut via requests.post() sur tous les endpoints."""
    for ep in endpoints:
        logger.info(f"\n=== Test HTTP brut pour {ep['name']} ===")
        
        url = f"{ep['api_base']}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ep['api_key']}"
        }
        payload = {
            "model": ep["model"],
            "messages": [
                {"role": "user", "content": "Bonjour, qui es-tu ?"}
            ],
            "max_completion_tokens": 50
        }
        
        logger.debug("  -> Envoi de la requête POST...")
        start_time = time.time()
        try:
            resp = requests.post(url, headers=headers, json=payload, timeout=30)
            elapsed_time = time.time() - start_time
            logger.debug(f"  -> Statut HTTP: {resp.status_code} (durée={elapsed_time:.2f}s)")
            
            # On essaie d'obtenir le JSON de la réponse
            try:
                resp_json = resp.json()
            except json.JSONDecodeError:
                logger.error(f"Réponse non-JSON:\n{resp.text[:200]}")
                continue
            
            # Affichage d’un extrait du JSON (en DEBUG, car potentiellement verbeux)
            logger.debug(f"Réponse (début): {json.dumps(resp_json, indent=2)[:1000]}...")

            # Nombre de tokens si success
            tokens_used = None
            if resp.status_code == 200:
                if "usage" in resp_json:
                    tokens_used = resp_json["usage"].get("total_tokens")
                logger.info(
                    f"[OK] Réponse HTTP 200 en {elapsed_time:.2f}s, "
                    f"Tokens utilisés={tokens_used if tokens_used else 'N/A'}"
                )
            else:
                # On log au niveau ERROR pour signifier un souci
                logger.error(
                    f"[ERREUR] HTTP {resp.status_code}, texte={resp_json.get('message', resp.text)}"
                )
        except requests.exceptions.Timeout:
            logger.error("Timeout après 30s.")
        except Exception as e:
            logger.exception(f"Exception lors de la requête: {str(e)}")

# On exécute le test brut
test_brut_endpoints()


09:41:03 [INFO] Local Llama - 
=== Test HTTP brut pour OpenAI ===
09:41:03 [DEBUG] Local Llama -   -> Envoi de la requête POST...
09:41:08 [DEBUG] Local Llama -   -> Statut HTTP: 200 (durée=5.20s)
09:41:08 [DEBUG] Local Llama - Réponse (début): {
  "id": "chatcmpl-B4kmwA6labaIiRNTBmR8hxGjGFUtp",
  "object": "chat.completion",
  "created": 1740472866,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Bonjour ! Je suis un assistant virtuel, con\u00e7u pour aider avec des informations et r\u00e9pondre \u00e0 vos questions. Comment puis-je vous aider aujourd'hui ?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 30,
    "total_tokens": 43,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens":

## 5) Test avec la librairie `openai`

On reproduit un appel classique OpenAI, 
mais en changeant `openai.api_base` et `openai.api_key` 
pour chaque endpoint.


In [10]:
from openai import OpenAI

def test_openai_chat(api_base, api_key, prompt, model):
    """
    Appel classique OpenAI, en utilisant la classe `OpenAI`
    et en gérant la journalisation via logger.
    """
    # Création du client OpenAI-compatible
    client = OpenAI(
        api_key=api_key,
        base_url=api_base
    )

    if not model:
        logger.error("[!] Modèle non défini.")
        raise ValueError("Modèle non défini")

    try:
        # Appel chat.completions
        logger.debug("Appel de client.chat.completions.create()...")
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_completion_tokens=500
        )
        content = response.choices[0].message.content
        tokens_used = response.usage.total_tokens if response.usage else None
        return content, tokens_used
    except Exception as e:
        logger.exception(f"Exception lors de l'appel OpenAI: {str(e)}")
        return None, None

def test_openai_endpoints():
    """Itère sur tous les endpoints et lance un prompt 'philosophie stoïcienne'."""
    for ep in endpoints:
        label_model = ep.get("model", "<aucun>")
        logger.info(f"\n=== Test openai pour {label_model} ({ep['name']}) ===")
        
        start_time = time.time()
        prompt = "Peux-tu résumer la philosophie stoïcienne en quelques lignes ?"
        content, tks = test_openai_chat(
            ep["api_base"],
            ep["api_key"],
            prompt,
            ep["model"]
        )
        elapsed_time = time.time() - start_time

        if content:
            logger.info(f" -> Réponse (début): {content[:200]}...")
            logger.info(f" -> Nb tokens: {tks}")
            logger.info(f" -> Temps écoulé: {elapsed_time:.2f} sec")
        else:
            logger.warning(" -> Pas de contenu (erreur ou exception).")

# On exécute le test
test_openai_endpoints()


09:41:27 [INFO] Local Llama - 
=== Test openai pour gpt-4o-mini (OpenAI) ===
09:41:27 [DEBUG] Local Llama - Appel de client.chat.completions.create()...
09:41:30 [INFO] Local Llama -  -> Réponse (début): La philosophie stoïcienne, fondée dans la Grèce antique par Zénon de Citium, met l'accent sur la maîtrise de soi, la vertu et la sagesse. Les stoïciens croient que le bonheur réside dans l'acceptation...
09:41:30 [INFO] Local Llama -  -> Nb tokens: 178
09:41:30 [INFO] Local Llama -  -> Temps écoulé: 3.38 sec
09:41:30 [INFO] Local Llama - 
=== Test openai pour Qwen/Qwen2.5-3B-Instruct-AWQ (Local Model - Micro) ===
09:41:30 [DEBUG] Local Llama - Appel de client.chat.completions.create()...
09:41:32 [INFO] Local Llama -  -> Réponse (début): La philosoph stoïcienne est fondamentalement axée sur la meilleure adaptation de l'homme au monde. Elle enseigne que l'acceptation de la vie telle qu'elle est, sans attentes excessives ou espoirs nuis...
09:41:32 [INFO] Local Llama -  -> Nb tokens: 227

## 6) Test avec Semantic Kernel (optionnel)

Exemple d'intégration avec [Semantic Kernel](https://github.com/microsoft/semantic-kernel).
On crée un Kernel, on y ajoute un service chat OpenAI-like avec l'endpoint souhaité, 
et on exécute un prompt simple.


In [11]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings
)
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments
from openai import AsyncOpenAI
import asyncio

async def test_semantic_kernel():
    """
    Exécute un prompt via Semantic Kernel pour chaque endpoint,
    et journalise les résultats en couleur via `logger`.
    """
    for ep in endpoints:
        model_id = ep.get("model")
        api_key = ep["api_key"]

        logger.info(f"=== Test Semantic Kernel pour endpoint='{ep['name']}', model='{model_id}' ===")

        kernel = sk.Kernel()
        async_client = AsyncOpenAI(api_key=api_key, base_url=ep["api_base"])

        kernel.add_service(
            OpenAIChatCompletion(
                service_id="default",
                ai_model_id=model_id,
                async_client=async_client
            )
        )
        logger.debug("Service OpenAI ajouté au Kernel.")

        prompt_template = "Explique ce qu'est l'apprentissage profond (deep learning) en 500 mots."

        exec_settings = OpenAIChatPromptExecutionSettings(
            service_id="default",
            ai_model_id=model_id,
            max_completion_tokens=500,
        )

        pt_config = PromptTemplateConfig(
            template=prompt_template,
            name="deepLearningFunction",
            template_format="semantic-kernel",
            input_variables=[],
            execution_settings=exec_settings,
        )

        func = kernel.add_function(
            function_name="deepLearningFunction",
            plugin_name="defaultPlugin",
            prompt_template_config=pt_config
        )

        try:
            logger.info("  -> Exécution en cours (Semantic Kernel)...")
            start_time = time.time()
            result = await kernel.invoke(func, KernelArguments())
            elapsed = time.time() - start_time

            # Comptage approximatif de tokens
            tokens_count = len(str(result).split())
            speed = tokens_count / elapsed if elapsed > 0 else 0

            logger.info(f"  -> Résultat (début): {str(result)[:400]}...")
            logger.info(f"  -> Durée: {elapsed:.2f}s, Tokens={tokens_count}, speed={speed:.2f} tok/s")
        except Exception as e:
            logger.exception(f"  [!] Erreur SK sur endpoint='{ep['name']}': {str(e)}")


import nest_asyncio
nest_asyncio.apply()

await test_semantic_kernel()


09:41:53 [INFO] Local Llama - === Test Semantic Kernel pour endpoint='OpenAI', model='gpt-4o-mini' ===
09:41:53 [DEBUG] Local Llama - Service OpenAI ajouté au Kernel.
09:41:53 [INFO] Local Llama -   -> Exécution en cours (Semantic Kernel)...
09:42:01 [INFO] Local Llama -   -> Résultat (début): L'apprentissage profond, ou deep learning en anglais, est une branche de l'intelligence artificielle et plus spécifiquement du machine learning (apprentissage automatique), qui vise à imiter le fonctionnement du cerveau humain pour analyser et interpréter des données complexes. Cette approche utilise des réseaux de neurones artificiels, qui sont des systèmes informatiques composés de noeuds (neuro...
09:42:01 [INFO] Local Llama -   -> Durée: 8.31s, Tokens=362, speed=43.54 tok/s
09:42:01 [INFO] Local Llama - === Test Semantic Kernel pour endpoint='Local Model - Micro', model='Qwen/Qwen2.5-3B-Instruct-AWQ' ===
09:42:01 [DEBUG] Local Llama - Service OpenAI ajouté au Kernel.
09:42:01 [INFO] Local Lla

## 9.1) Test du Function/Tool Calling sur chaque endpoint

Avec vLLM, lorsqu’on démarre les containers avec `--enable-auto-tool-choice` et un `--tool-call-parser` adéquat,
le modèle peut déclencher automatiquement un « tool call » s’il juge qu’un outil est pertinent.  
On doit alors inclure un paramètre `tools` dans la requête, et indiquer `tool_choice="auto"` (ou un nom de fonction précis).

**Note** : Pour exécuter concrètement la fonction côté client Python, on doit définir une fonction Python qui correspond,
et réinjecter manuellement le résultat dans la conversation.  
Voici un exemple simplifié : on va appeler un `get_weather(location, unit)` sur *tous* les endpoints.


In [16]:
import json
import openai
from openai import OpenAI

def get_weather(location: str, unit: str):
    """Exemple de fonction locale pour la météo."""
    return f"Simulation: Météo à {location}, unité={unit}, ciel dégagé."

def test_tool_calling():
    """
    Test du Function/Tool Calling pour chaque endpoint, en mode auto (tool_choice='auto').
    On journalise les étapes et le résultat.
    """
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Obtenir la météo pour un lieu donné",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "Ex: 'Paris, France'"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                    },
                    "required": ["location", "unit"]
                }
            }
        }
    ]

    user_message = "Bonjour, est-ce que tu peux me donner la météo pour Marseille en celsius ?"

    for ep in endpoints:
        logger.info(f"=== Test Tool Calling sur endpoint '{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            logger.debug("Appel chat.completions avec tool_choice='auto'")
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": user_message}],
                tools=tools,
                tool_choice="auto",
            )
        except Exception as ex:
            logger.error(f"[!] Erreur lors de l'appel sur endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        text_content = msg.content if msg.content else ""
        tool_calls = msg.tool_calls

        logger.debug(f"Réponse textuelle (début): {text_content[:400]!r}")
        logger.debug(f"Contenu complet de la réponse: {json.dumps(response.to_dict(), indent=2)}")

        if tool_calls:
            logger.info(f"  -> {len(tool_calls)} tool_call(s) détecté(s) sur endpoint='{ep['name']}'.")
            for call in tool_calls:
                func_name = call.function.name
                args_str  = call.function.arguments or "{}"
                args_dict = json.loads(args_str)

                logger.info(f"     Fonction appelée: {func_name} | arguments={args_dict}")

                # Exécution locale simulée
                if func_name == "get_weather":
                    result = get_weather(**args_dict)
                    logger.info(f"     => Résultat simulé: {result}")
                else:
                    logger.warning(f"     => Fonction inconnue: {func_name}")
        else:
            logger.warning("  -> Aucun tool_call détecté dans la réponse.")

# On l’exécute
test_tool_calling()


09:46:35 [INFO] Local Llama - === Test Tool Calling sur endpoint 'OpenAI' ===
09:46:35 [DEBUG] Local Llama - Appel chat.completions avec tool_choice='auto'
09:46:36 [DEBUG] Local Llama - Réponse textuelle (début): ''
09:46:36 [DEBUG] Local Llama - Contenu complet de la réponse: {
  "id": "chatcmpl-B4ksESBX7nMT1B8H8TJ5rNJ2jfkyx",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_mAJrbi6cFUwY9plMzOzcpsap",
            "function": {
              "arguments": "{\"location\":\"Marseille, France\",\"unit\":\"celsius\"}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1740473194,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "

## 9.2) Test du mode « Reasoning Outputs »

Certains modèles (p. ex. DeepSeek R1) sont lancés avec `--enable-reasoning --reasoning-parser deepseek_r1`.
Cela permet de renvoyer, en plus du `content` final, un champ `reasoning_content` qui détaille la chaîne de raisonnement.

Voici un exemple d’appel sur *tous* les endpoints (certains n’auront pas de champ `reasoning_content` si le modèle ne supporte pas le raisonnement).


In [15]:
def test_reasoning():
    """
    Test du mode reasoning_output (champ 'reasoning_content') 
    s'il est activé sur certains modèles. 
    """
    question = "Combien font 253 * 73 - 287 ?"

    for ep in endpoints:
        logger.info(f"=== Test Reasoning Output sur endpoint='{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": question}]
            )
        except Exception as ex:
            logger.error(f"  [!] Erreur lors de l'appel endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        reasoning_part = getattr(msg, "reasoning_content", None)
        final_content = msg.content or ""

        logger.debug(f"  -> Réponse (finale) : {final_content[:250]}")
        if reasoning_part:
            logger.info(f"  -> Raisonnement   : {reasoning_part[:250]}")
        else:
            logger.warning("  -> Pas de 'reasoning_content' pour ce modèle/parsing.")

# Lancement
test_reasoning()


09:46:02 [INFO] Local Llama - === Test Reasoning Output sur endpoint='OpenAI' ===
09:46:04 [DEBUG] Local Llama -   -> Réponse (finale) : Pour résoudre l'expression \( 253 \times 73 - 287 \), commençons par effectuer la multiplication :

\[ 253 \times 73 = 18469 \]

Ensuite, nous soustrayons 287 :

\[ 18469 - 287 = 18182 \]

Donc, \( 253 \times 73 - 287 = 18182 \).
09:46:04 [WARNING] Local Llama -   -> Pas de 'reasoning_content' pour ce modèle/parsing.
09:46:04 [INFO] Local Llama - === Test Reasoning Output sur endpoint='Local Model - Micro' ===
09:46:05 [DEBUG] Local Llama -   -> Réponse (finale) : Pour résoudre cette équation, on procède comme suit :

1) Tout d'abord, on calcule le produit 253 * 73.

2) Ensuite on retire 287 du résultat obtenu.

Alors, suivant les règles de l'arithmétique classique (effectuant les opérations de gauche à droite
09:46:05 [WARNING] Local Llama -   -> Pas de 'reasoning_content' pour ce modèle/parsing.
09:46:05 [INFO] Local Llama - === Test Reasoning Outpu

## 7) Benchmark final (avec journaux réguliers)

Cette étape exécute un warm-up + N itérations par endpoint.
On calcule ensuite la vitesse tokens/s. 

**Important** : Le prompt est un peu plus long, et la génération peut 
prendre du temps selon la taille du modèle ou la quantization.

Pour ne pas paraître figé, on ajoute des `print` avant et après l'appel, 
pour indiquer la progression.


In [17]:
import sys
import openai
from openai import OpenAI

# Paramètre: combien de fois on répète la requête pour la mesure.
N_REPEATS = 1
all_results = []

def query_once(api_base, api_key, prompt, model=None):
    """
    Appel unique via la nouvelle API (client = OpenAI(...)) + log.
    Retourne (elapsed_seconds, total_tokens).
    """
    client = OpenAI(api_key=api_key, base_url=api_base)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    start_time = time.time()
    logger.debug("      ... appel client.chat.completions.create() en cours ...")
    try:
        resp = client.chat.completions.create(
            messages=messages,
            model=model,
            max_completion_tokens=1000,
            stream=False,
        )
        elapsed = time.time() - start_time

        tokens = resp.usage.total_tokens if resp.usage else None
        return elapsed, tokens
    except Exception as e:
        logger.error(f"      [!] Exception: {e}")
        return None, None

def run_benchmark(label, prompt, repeats=N_REPEATS):
    """
    1) Warm-up (unique appel) + N appels chronométrés par endpoint.
    2) On log avant/après chaque appel via logger.
    3) On stocke les métriques dans all_results.
    """
    global all_results

    for ep in endpoints:
        logger.info(f"=== Benchmark: endpoint={ep['name']}, label={label} ===")

        # WARM-UP
        logger.info("   (warm-up) Lancement d'un appel simple.")
        w_elapsed, w_tokens = query_once(
            ep["api_base"], ep["api_key"],
            "Warm up. Ignorez ce message.",
            model=ep.get("model", None)
        )
        if w_elapsed is None:
            logger.warning(f"   [!] Warm-up échoué => skip {ep['name']}.")
            continue

        total_time = 0.0
        total_tokens = 0
        success_count = 0

        logger.info(f"   (benchmark) On va faire {repeats} itérations.")
        for i in range(repeats):
            logger.info(f"   -> Iteration {i+1}/{repeats}")
            e, tks = query_once(
                ep["api_base"], ep["api_key"],
                prompt,
                model=ep.get('model', None)
            )
            if e is None:
                logger.error("      Echec de l'appel, on continue avec le suivant.")
                continue
            logger.info(f"      => Durée: {e:.2f}s, tokens={tks}")
            total_time += e
            if tks is not None:
                total_tokens += tks
            success_count += 1

        if success_count == 0:
            logger.warning("   [!] Aucune itération réussie pour ce endpoint.")
            continue

        avg_time = total_time / success_count
        tok_per_sec = 0.0
        if total_time > 0:
            tok_per_sec = total_tokens / total_time

        res = {
            "label": label,
            "endpoint": ep["name"],
            "repeats": repeats,
            "success_count": success_count,
            "total_time_s": total_time,
            "avg_time_s": avg_time,
            "total_tokens": total_tokens,
            "tokens_per_sec": tok_per_sec
        }
        all_results.append(res)
        logger.info(
            f"   => {ep['name']} OK: {success_count}/{repeats} calls, "
            f"avg_time={avg_time:.2f}s, speed={tok_per_sec:.2f} tok/s"
        )

    logger.info(f"[INFO] Fin du benchmark pour label='{label}'.")

# Exemple d’appel:
label_run = "auto_benchmark1"
USER_PROMPT = (
    "Rédige un texte d'environ 1000 mots sur l'IA, "
    "en évoquant l'apprentissage machine, les grands modèles de langage, "
    "et quelques perspectives d'évolution."
)

run_benchmark(label_run, USER_PROMPT, repeats=1)

logger.info(f"\n=== Résultats de ce premier benchmark (label='{label_run}') ===")
for r in all_results:
    if r["label"] == label_run:
        logger.info(
            f"{r['endpoint']} -> {r['success_count']}/{r['repeats']} ok, "
            f"avg time={r['avg_time_s']:.2f}s, speed={r['tokens_per_sec']:.2f} tok/s"
        )



09:47:18 [INFO] Local Llama - === Benchmark: endpoint=OpenAI, label=auto_benchmark1 ===
09:47:18 [INFO] Local Llama -    (warm-up) Lancement d'un appel simple.
09:47:18 [DEBUG] Local Llama -       ... appel client.chat.completions.create() en cours ...
09:47:20 [INFO] Local Llama -    (benchmark) On va faire 1 itérations.
09:47:20 [INFO] Local Llama -    -> Iteration 1/1
09:47:21 [DEBUG] Local Llama -       ... appel client.chat.completions.create() en cours ...
09:47:34 [INFO] Local Llama -       => Durée: 13.50s, tokens=1053
09:47:34 [INFO] Local Llama -    => OpenAI OK: 1/1 calls, avg_time=13.50s, speed=78.01 tok/s
09:47:34 [INFO] Local Llama - === Benchmark: endpoint=Local Model - Micro, label=auto_benchmark1 ===
09:47:34 [INFO] Local Llama -    (warm-up) Lancement d'un appel simple.
09:47:34 [DEBUG] Local Llama -       ... appel client.chat.completions.create() en cours ...
09:47:35 [INFO] Local Llama -    (benchmark) On va faire 1 itérations.
09:47:35 [INFO] Local Llama -    -> I

## 8) Relancer le benchmark après mise à jour des containers

Pour passer de Oobabooga à vLLM (ou inversement), 
il faut arrêter le premier groupe de containers 
et démarrer le second (avec `docker-compose.*.yml`).

Ensuite, on ré-exécute ce notebook (ou au moins les cellules de configuration + benchmark).

Ici, on propose un prompt qui demande si tu veux relancer un benchmark 
avec un nouveau label.


In [ ]:
do_rerun = input("Voulez-vous exécuter un nouveau benchmark ? (y/n) ").strip().lower()

if do_rerun == "y":
    rename_label = input("Entrez un label pour le 1er benchmark (actuellement 'auto_benchmark1'): ").strip()
    for r in all_results:
        if r["label"] == "auto_benchmark1":
            r["label"] = rename_label
    second_label = input("Entrez un label pour le 2e benchmark : ").strip()
    run_benchmark(second_label, USER_PROMPT, repeats=N_REPEATS)

logger.info("\n=== Récapitulatif complet de tous les benchmarks ===")
for r in all_results:
    logger.info(
        f"{r['label']} | {r['endpoint']} -> {r['success_count']}/{r['repeats']} ok, "
        f"time={r['avg_time_s']:.2f}s, speed={r['tokens_per_sec']:.2f} tok/s"
    )


## 9) Test de traitement parallèle (batching)

Dans cette cellule, nous allons :
- Définir un nombre de requêtes à envoyer en parallèle (`N_PARALLEL`).
- Pour chaque endpoint, lancer ces requêtes en **concurrence**.
- Mesurer le temps total écoulé et le nombre total de tokens.
- Calculer la **vitesse globale** de traitement (tokens / seconde) lorsque plusieurs requêtes arrivent simultanément.

**vLLM** est réputé supporter le batching token-level et donc bénéficier d'une meilleure latence moyenne et d'un meilleur débit lorsqu'il y a plusieurs requêtes en parallèle.


In [18]:
import aiohttp
import asyncio

N_PARALLEL = 25
PARALLEL_PROMPT = (
    "Bonjour, ceci est un test de requêtes parallèles. "
    "Peux-tu me donner quelques idées créatives pour un week-end ?"
)

async def async_chat_completion(api_base: str, api_key: str, model: str, prompt: str):
    url = f"{api_base}/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 200
    }

    async with aiohttp.ClientSession() as session:
        start_t = time.time()
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                elapsed = time.time() - start_t
                if resp.status == 200:
                    data = await resp.json()
                    tokens = None
                    if "usage" in data and data["usage"].get("total_tokens"):
                        tokens = data["usage"]["total_tokens"]
                    return (elapsed, tokens)
                else:
                    return (None, None)
        except Exception as e:
            logger.error(f"[!] Exception asynchrone: {e}")
            return (None, None)

async def run_parallel_test(endpoint, n_parallel, prompt):
    api_base = endpoint["api_base"]
    api_key  = endpoint["api_key"]
    model    = endpoint.get("model", None)

    tasks = []
    for _ in range(n_parallel):
        tasks.append(asyncio.create_task(async_chat_completion(api_base, api_key, model, prompt)))

    start_time = time.time()
    results = await asyncio.gather(*tasks)
    total_time = time.time() - start_time

    nb_ok = 0
    sum_tokens = 0
    for (elapsed, tokens) in results:
        if elapsed is not None and tokens is not None:
            nb_ok += 1
            sum_tokens += tokens

    return {
        "endpoint": endpoint["name"],
        "n_req": n_parallel,
        "n_ok": nb_ok,
        "total_time_s": total_time,
        "sum_tokens": sum_tokens
    }

async def parallel_benchmark(endpoints_list, n_parallel, prompt, use_random_prefix=True):
    summary = []
    logger.info(f"==== Lancement du test parallèle: {n_parallel} requêtes simultanées par endpoint ====")
    
    for ep in endpoints_list:
        logger.info(f"=== Parallel test sur '{ep['name']}' ===")
        
        if use_random_prefix:
            prefix = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=5))
            modified_prompt = f"{prefix} {prompt}"
        else:
            modified_prompt = prompt
        
        res = await run_parallel_test(ep, n_parallel, modified_prompt)

        nb_ok = res["n_ok"]
        total_time = res["total_time_s"]
        sum_tokens = res["sum_tokens"]
        speed = sum_tokens / total_time if total_time > 0 else 0

        logger.info(f"  -> {nb_ok}/{n_parallel} appels OK")
        logger.info(f"  -> Durée totale (concurrente): {total_time:.2f} s")
        logger.info(f"  -> Tokens cumulés: {sum_tokens}")
        logger.info(f"  -> Vitesse globale: {speed:.2f} tok/s")

        summary.append(res)

    logger.info("\n=== Récapitulatif du test parallèle ===")
    for s in summary:
        speed = 0.0
        if s["total_time_s"] > 0:
            speed = s["sum_tokens"] / s["total_time_s"]
        logger.info(
            f"{s['endpoint']}: {s['n_ok']}/{s['n_req']} ok, "
            f"total_time={s['total_time_s']:.2f}s, speed={speed:.2f} tok/s"
        )

import nest_asyncio
import random
nest_asyncio.apply()

await parallel_benchmark(endpoints, N_PARALLEL, PARALLEL_PROMPT)


09:48:54 [INFO] Local Llama - ==== Lancement du test parallèle: 25 requêtes simultanées par endpoint ====
09:48:54 [INFO] Local Llama - === Parallel test sur 'OpenAI' ===
09:48:57 [INFO] Local Llama -   -> 25/25 appels OK
09:48:57 [INFO] Local Llama -   -> Durée totale (concurrente): 3.45 s
09:48:57 [INFO] Local Llama -   -> Tokens cumulés: 5900
09:48:57 [INFO] Local Llama -   -> Vitesse globale: 1708.52 tok/s
09:48:57 [INFO] Local Llama - === Parallel test sur 'Local Model - Micro' ===
09:49:01 [INFO] Local Llama -   -> 25/25 appels OK
09:49:01 [INFO] Local Llama -   -> Durée totale (concurrente): 4.10 s
09:49:01 [INFO] Local Llama -   -> Tokens cumulés: 6600
09:49:01 [INFO] Local Llama -   -> Vitesse globale: 1609.01 tok/s
09:49:01 [INFO] Local Llama - === Parallel test sur 'Local Model - Mini' ===
09:49:07 [INFO] Local Llama -   -> 25/25 appels OK
09:49:07 [INFO] Local Llama -   -> Durée totale (concurrente): 6.10 s
09:49:07 [INFO] Local Llama -   -> Tokens cumulés: 6575
09:49:07 [I

## 10) Test de parallélisme global (MAJ) : mesures de débits individuels et ordre d’exécution aléatoire

Ici, nous souhaitons :
1. **Mesurer** le débit (tokens/s) **individuellement** pour chaque endpoint.
2. **Lancer** toutes les requêtes (pour tous les endpoints) **en ordre aléatoire**, sans rajouter de délais artificiels.

**Principe** :
- On construit d’abord **la liste** complète des appels (ex. `N_PARALLEL_GLOBAL` requêtes pour chaque endpoint).
- On associe à chaque appel l’endpoint correspondant, puis on **randomise** l’ordre de cette liste.
- On déclenche **en simultané** l’ensemble des requêtes (via `asyncio.gather`).
- Après exécution, on calcule :
  - **Durée totale** (début → fin) pour l’ensemble des requêtes,
  - **Résultats individuels** (tokens cumulés par endpoint, nombre de requêtes OK, etc.),
  - **Débit** de chaque endpoint : (somme des tokens pour cet endpoint) / (durée totale),
  - **Débit global** : (somme de tous les tokens) / (durée totale).


In [26]:
import aiohttp
import asyncio
import random
import time

N_PARALLEL_GLOBAL = 25  # Nombre de requêtes à envoyer par endpoint
GLOBAL_PROMPT = (
    "Bonjour, ceci est un test de parallélisme global. "
    "Peux-tu me détailler en 500 mots les avantages et inconvénients de travailler "
    "avec plusieurs grands modèles (Llama, Qwen, GPT, etc.) en parallèle sur un même serveur ?"
)

async def async_chat_completion(api_base: str, api_key: str, model: str, prompt: str):
    """
    Effectue une requête unique (POST /chat/completions) en asynchrone.
    Retourne un tuple (start_t, end_t, tokens, success).
      - start_t  : l'instant du début effectif (time.time()) juste avant l'appel
      - end_t    : l'instant de fin (time.time()) juste après la réception de la réponse
      - tokens   : nombre de tokens dans la réponse (None si échec)
      - success  : booléen (True si statut=200 et parse JSON OK)
    """
    # On note l'instant de démarrage avant de créer la session et d'envoyer la requête
    start_t = time.time()

    url = f"{api_base}/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 150
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                end_t = time.time()
                if resp.status == 200:
                    data = await resp.json()
                    tokens = data.get("usage", {}).get("total_tokens", None)
                    return (start_t, end_t, tokens, True)
                else:
                    return (start_t, end_t, None, False)
        except Exception as e:
            logger.error(f"[!] Exception asynchrone: {e}")
            end_t = time.time()
            return (start_t, end_t, None, False)

async def run_full_parallel_on_all_endpoints(endpoints, n_parallel, prompt):
    """
    Lance n_parallel requêtes pour chaque endpoint (donc total = n_parallel * len(endpoints)),
    en un seul grand batch concurrent *et dans un ordre aléatoire*.

    Retourne un dict contenant :

    - global_min_start  : le plus petit start_t (sur toutes les requêtes, tous endpoints)
    - global_max_end    : le plus grand end_t   (sur toutes les requêtes)
    - n_req_total       : total de requêtes
    - n_ok_global       : total de requêtes ayant répondu 200 OK
    - sum_tokens_global : total de tokens (toutes requêtes OK)
    - stats_endpoints   : dict par endpoint, contenant :
         {
           "calls": int,
           "ok": int,
           "sum_tokens": int,
           "min_start": float,
           "max_end": float
         }
      (on calcule ensuite un débit = sum_tokens / (max_end - min_start))
    """

    # Prépare tous les appels (endpoint, prompt)
    tasks_info = []
    for ep in endpoints:
        for _ in range(n_parallel):
            # Optionnel : préfixe aléatoire pour "casser" un éventuel cache
            prefix = "".join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZ", k=3))
            modified_prompt = f"{prefix} {prompt}"
            tasks_info.append((ep, modified_prompt))

    # Mélange l’ordre des requêtes
    random.shuffle(tasks_info)

    # Transforme chaque (ep, prompt) en coroutine
    coroutines = []
    for (ep, pr) in tasks_info:
        coroutines.append(async_chat_completion(ep["api_base"], ep["api_key"], ep["model"], pr))

    # Exécution en parallèle
    results = await asyncio.gather(*coroutines)

    # Rassemble toutes les stats
    # On calcule le min de start_t et le max de end_t global
    all_starts = []
    all_ends = []

    stats_per_endpoint = {}
    for ep in endpoints:
        stats_per_endpoint[ep["name"]] = {
            "calls": 0,
            "ok": 0,
            "sum_tokens": 0,
            "min_start": float("inf"),
            "max_end": float("-inf"),
        }

    global_ok = 0
    global_tokens = 0

    for i, (start_t, end_t, tokens, success) in enumerate(results):
        ep_name = tasks_info[i][0]["name"]
        ep_stats = stats_per_endpoint[ep_name]
        ep_stats["calls"] += 1

        # On enrichit le min/max local à l'endpoint
        if start_t < ep_stats["min_start"]:
            ep_stats["min_start"] = start_t
        if end_t > ep_stats["max_end"]:
            ep_stats["max_end"] = end_t

        # Idem pour le global
        all_starts.append(start_t)
        all_ends.append(end_t)

        # On comptabilise tokens si success
        if success and tokens is not None:
            ep_stats["ok"] += 1
            ep_stats["sum_tokens"] += tokens
            global_ok += 1
            global_tokens += tokens

    # min/max global
    global_min_start = min(all_starts) if all_starts else None
    global_max_end = max(all_ends) if all_ends else None

    summary = {
        "global_min_start": global_min_start,
        "global_max_end": global_max_end,
        "n_req_total": len(results),
        "n_ok_global": global_ok,
        "sum_tokens_global": global_tokens,
        "stats_endpoints": stats_per_endpoint
    }
    return summary


# --- Lancement effectif du test ---
logger.info(f"=== Lancement du test de parallélisme global sur {len(endpoints)} endpoints ===")
logger.info(f"   -> {N_PARALLEL_GLOBAL} requêtes par endpoint (ordre aléatoire).")

summary_global = await run_full_parallel_on_all_endpoints(endpoints, N_PARALLEL_GLOBAL, GLOBAL_PROMPT)

# Récapitulatif
global_min_start = summary_global["global_min_start"]
global_max_end = summary_global["global_max_end"]
n_req_total = summary_global["n_req_total"]
n_ok_global = summary_global["n_ok_global"]
sum_tokens_global = summary_global["sum_tokens_global"]

logger.info("")
logger.info("=== Résultats du test global (tous endpoints en même temps) ===")
logger.info(f"  -> Nombre total de requêtes : {n_req_total}")
logger.info(f"  -> Nombre de requêtes OK   : {n_ok_global}")

if global_min_start is not None and global_max_end is not None:
    global_duration = global_max_end - global_min_start
    logger.info(f"  -> Fenêtre de temps (global) : {global_duration:.2f} s")
    logger.info(f"  -> Cumul de tokens (global)  : {sum_tokens_global}")
    if global_duration > 0:
        global_speed = sum_tokens_global / global_duration
        logger.info(f"  -> Débit global (tous endpoints) : {global_speed:.2f} tok/s")

logger.info("\n=== Détails par endpoint ===")
for ep_name, ep_stats in summary_global["stats_endpoints"].items():
    calls = ep_stats["calls"]
    ok = ep_stats["ok"]
    sum_tks = ep_stats["sum_tokens"]
    ep_min_start = ep_stats["min_start"]
    ep_max_end   = ep_stats["max_end"]

    logger.info(f"- {ep_name} : {ok}/{calls} OK, tokens={sum_tks}")
    if ok > 0 and ep_min_start < ep_max_end:  # au moins 1 requête
        ep_concurrency_window = ep_max_end - ep_min_start
        # 'ep_concurrency_window' = fenêtrage du 1er démarrage -> dernier aboutissement
        speed_ep = sum_tks / ep_concurrency_window if ep_concurrency_window > 0 else 0
        logger.info(f"    => Fenêtre concurrency = {ep_concurrency_window:.2f}s")
        logger.info(f"    => Débit effectif ~ {speed_ep:.2f} tok/s")
    else:
        logger.info("    => Pas de requêtes OK ou pas de fenêtre exploitable.")


10:29:48 [INFO] Local Llama - === Lancement du test de parallélisme global sur 5 endpoints ===
10:29:48 [INFO] Local Llama -    -> 25 requêtes par endpoint (ordre aléatoire).
10:30:23 [INFO] Local Llama - 
10:30:23 [INFO] Local Llama - === Résultats du test global (tous endpoints en même temps) ===
10:30:23 [INFO] Local Llama -   -> Nombre total de requêtes : 125
10:30:23 [INFO] Local Llama -   -> Nombre de requêtes OK   : 125
10:30:23 [INFO] Local Llama -   -> Fenêtre de temps (global) : 35.15 s
10:30:23 [INFO] Local Llama -   -> Cumul de tokens (global)  : 29476
10:30:23 [INFO] Local Llama -   -> Débit global (tous endpoints) : 838.57 tok/s
10:30:23 [INFO] Local Llama - 
=== Détails par endpoint ===
10:30:23 [INFO] Local Llama - - OpenAI : 25/25 OK, tokens=5273
10:30:23 [INFO] Local Llama -     => Fenêtre concurrency = 5.30s
10:30:23 [INFO] Local Llama -     => Débit effectif ~ 994.24 tok/s
10:30:23 [INFO] Local Llama - - Local Model - Micro : 25/25 OK, tokens=6201
10:30:23 [INFO] Lo